# BC5 group S

Rever este texto e confirmar se acham importante deixar ou nao


During the computing process we need to creat various load systems to save time and make easier to re-start working where we have stoped before

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster.elbow import KElbowVisualizer
from sklearn.mixture import GaussianMixture
from IPython.display import YouTubeVideo
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram


from sklearn import preprocessing
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler, normalize


#  1- Load the dataset

Because of the size and the weight of the dataset we had to load  the dataset in chunks of 10M Rows
After the creation of each chunk we upload each chunk to a different csv fille to be easier to read the filles in the future

To understand what could be donne with the dataset we load him with only 100 rows

In [ ]:
pd.read_csv(r'C:\Users\classroom\Documents\b5data\data.csv', nrows=100)

In the previows table it's visible that 7/9 columns have information that will not be needed to the analisies and that make the dataset heavy.
So we created a function that will clean those columms and make the dataset much lighter.
To avoid memory problems we will run this cleaning function in the chunks and not in the all datset together.

In [ ]:
def pre_process(ds):
    ds['ProductFamily_ID'] = ds['ProductFamily_ID'].str.split('_').str[1]
    ds['ProductCategory_ID'] = ds['ProductCategory_ID'].str.split('_').str[1]
    ds['ProductBrand_ID'] = ds['ProductBrand_ID'].str.split('_').str[1]
    ds['ProductName_ID'] = ds['ProductName_ID'].str.split('_').str[1]
    ds['ProductPackSKU_ID'] = ds['ProductPackSKU_ID'].str.split('_').str[1]
    ds['Point-of-Sale_ID'] = ds['Point-of-Sale_ID'].str.split('_').str[1]
    ds['Measures'] = ds['Measures'].str.split(' ').str[1]
    ds['Date'] = pd.to_datetime(ds['Date'])
    dS = ds.sort_values(by='Date')

In [ ]:
for no in range(1,20):
    df = pd.read_csv('chunkfile' + str(no) + '.csv')
    pre_process(df)
    df.to_csv('chunk_after_preproc' + str(no) + '.csv', index=False)

In [ ]:
#correr mais tarde para testar
a1 = pd.read_csv('chunk_after_preproc1.csv')

for no in range(2,20):
    a1 = a1.append(pd.read_csv('chunk_after_preproc' + str(no) + '.csv'))
    


a1.to_csv('merge1', index=False)    


# Preprocessing

In [ ]:
path = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/MindOverData_RetailChallenge.csv'
pathPP = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/values_units.csv'

In [ ]:
data =  pd.read_csv(path, nrows=1999998)

In [ ]:
#PreProcessing - removing excessive letters from the variables

data['ProductFamily_ID'] = data['ProductFamily_ID'].str.split('_').str[1]
data['ProductCategory_ID'] = data['ProductCategory_ID'].str.split('_').str[1]
data['ProductBrand_ID'] = data['ProductBrand_ID'].str.split('_').str[1]
data['ProductName_ID'] = data['ProductName_ID'].str.split('_').str[1]
data['ProductPackSKU_ID'] = data['ProductPackSKU_ID'].str.split('_').str[1]
data['Point-of-Sale_ID'] = data['Point-of-Sale_ID'].str.split('_').str[1]

data['Measures'] = data['Measures'].str.split(' ').str[1]

data.head(5)

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
#data = data.sort_values(by='Date')

In [ ]:
# Creating a variable with the day of the week of each purchase

data['Day of the Week'] = data['Date'].dt.dayofweek

In [ ]:
# Creating a variable with the correspondent quarter

data['Quarter'] = data['Date'].dt.quarter

In [ ]:
# Splitting the dataframe based on Units/ Values

data_units = data[data['Measures'] == 'units']
data_values = data[data['Measures'] == 'values']

In [ ]:
data_values = data_values.assign(Units=data_units['Value'].values)
data_values = data_values.drop(columns='Measures')

In [ ]:
#data_values.to_csv('values_units.csv.gz', compression='gzip')

# EDA

##### Loading the preprocessed dataset

In [ ]:
dataPP =  pd.read_csv(pathPP,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str})

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="Day of the Week", data=dataPP, color='darkblue')
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of the week', fontsize=12)
plt.title("Frequency of purchase by day of the week", fontsize=15)
plt.show()

In [ ]:
cnt = dataPP['ProductName_ID'].value_counts()[:10]
plt.figure(figsize=(12,8))
sns.barplot(cnt.values, cnt.index, alpha=0.8, color='gray')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Product ID', fontsize=12)
plt.title("Top 10 products", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(dataPP['Value'], color='green')
plt.xlabel('Value', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.show()

## Quarterly analysis

###### January, February, and March (Q1)
###### April, May, and June (Q2)
###### July, August, and September (Q3)
###### October, November, and December (Q4)

In [ ]:
fig = px.line(dataPP.sort_values(by='Date'), x="Date", y="Value")
fig.show()

In [ ]:
fig = px.bar(data_values, x="total_bill", y="day", orientation='h')
fig.show()

In [ ]:
# Further exploration was done using Power BI

#### Association rules

In [ ]:
col_list = ["ProductFamily_ID", "Point-of-Sale_ID","ProductName_ID","Units","Date","Quarter"]
dataPP =  pd.read_csv(pathPP, usecols=col_list, nrows=1000000,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str, 'ProductName_ID': str})

In [ ]:
dataPP['Date']= pd.to_datetime(dataPP['Date'])
dataPP['year'] = pd.DatetimeIndex(dataPP['Date']).year

In [ ]:
dataPP['Date'] = dataPP['Date'].astype(str)

dataPP['TID'] = dataPP['ProductFamily_ID'] + dataPP['Point-of-Sale_ID'] + ' ' + dataPP['Date']

In [ ]:
def mba(pos, qt, yr):
    fam =  dataPP[(dataPP['Point-of-Sale_ID'] == pos) & (dataPP['Quarter'] == qt) & (dataPP['year'] == yr)]
    fam = fam[['ProductName_ID','TID']]
    fam_pivot = pd.pivot_table(fam, index='TID', columns='ProductName_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
    frequent_itemsets = apriori(fam_pivot, min_support=0.05, use_colnames=True)
    print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))
    rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.50)
    rulesConfidence.sort_values(by='confidence', ascending=False, inplace=True)
    print(rulesConfidence.head())
    return 

In [ ]:
mba('1',1,2016)

In [ ]:
# Selecionar Pos / Quarter / Year
fam =  dataPP[(dataPP['Point-of-Sale_ID'] == '2') & (dataPP['Quarter'] == 1) & (dataPP['year'] == 2016)]

In [ ]:
fam = fam[['ProductName_ID','TID']]

In [ ]:
fam_pivot = pd.pivot_table(fam, index='TID', columns='ProductName_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)

In [ ]:
#fam.to_csv('fam_sample.csv.gz', compression='gzip')

In [ ]:
frequent_itemsets = apriori(fam_pivot, min_support=0.05, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head(10)

In [ ]:
# Generate the association rules - by confidence
rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.50)
rulesConfidence.sort_values(by='confidence', ascending=False, inplace=True)
rulesConfidence.head()

In [ ]:
# Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rulesLift.sort_values(by='lift', ascending=False, inplace=True)
rulesLift.head()

In [ ]:
data_scatter = dict(type='scatter',
    y=rulesConfidence['confidence'][:10],
    x=rulesConfidence['support'][:10],
    #text=rulesLift.consequents,
    #mode='markers',
    #marker=dict(
    #size=rulesLift['support'],
    hovertemplate='Lift: ' + rulesLift["lift"].astype(str) + '<br>'+
                    'Confidence: ' + rulesLift['confidence'].astype(str) + '<br>'+
                    'Support: ' + rulesLift['support'].astype(str) + '<br>'+    
                    'Antecedents: ' + rulesLift['antecedents'].astype(str) + '<br>'+
                    'Consequents: ' + rulesLift['consequents'].astype(str) + '<br>'
    '<extra></extra>',
    mode='markers',
    marker=dict(size=8,
                color=rulesLift['lift'],
                colorscale='dense',
                showscale=True,
                line_width=2),
    )


fig = go.Figure(data=data_scatter)
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,10)) 
sns.scatterplot(data=rulesLift, x="confidence", y="lift", hue="support")

# Clustering 

Clustering Functions

In [ ]:
def number_clusters(df,cluster_model):
    metrics=['silhouette','distortion','calinski_harabasz']
    model=cluster_model
    for score in metrics:
        plt.figure()
        plt.ylabel(score)
        plt.xlabel('Number of clusters')
        Visualizer=KElbowVisualizer(model,metric=score).fit(df)
        plt.show

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=-20)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
def gmm_config(data,limit_nrcomponents):
    bic_values =[]
    aic_values =[]
    types_covariance=['full','diag','spherical']
    for i in types_covariance:
        n_components = np.arange(1, limit_nrcomponents)
        models = [GaussianMixture(n, covariance_type=i, n_init=10, random_state=1).fit(data)
                  for n in n_components]
        bic_values.append([m.bic(data) for m in models])
        aic_values.append([m.aic(data) for m in models])
    #barplot for BIC
    bic=plt.figure(1)
    X = np.arange(1,limit_nrcomponents)
    plt.bar(X + 0.00, bic_values[0], label='full' ,color = 'b', width = 0.3)
    plt.bar(X + 0.25, bic_values[1], label='diag',color = 'g', width = 0.3)
    plt.bar(X + 0.50, bic_values[2], label='spherical',color = 'r', width = 0.3)
    plt.xlabel('number of components')
    plt.ylabel('BIC')
    plt.legend(loc='best')
   #barplot for AIC
    aic=plt.figure(2)
    X = np.arange(1,limit_nrcomponents)
    plt.bar(X + 0.00, aic_values[0], label='full' ,color = 'b', width = 0.3)
    plt.bar(X + 0.25, aic_values[1], label='diag',color = 'g', width = 0.3)
    plt.bar(X + 0.50, aic_values[2], label='spherical',color = 'r', width = 0.3)
    plt.xlabel('number of components')
    plt.ylabel('AIC')
    plt.legend(loc='best')
    bic.show
    aic.show

In [ ]:
def get_r2_hc(df, link_method, max_nclus, min_nclus=1, dist="euclidean"):
    """This function computes the R2 for a set of cluster solutions given by the application of a hierarchical method.
    The R2 is a measure of the homogenity of a cluster solution. It is based on SSt = SSw + SSb and R2 = SSb/SSt. 
    
    Parameters:
    df (DataFrame): Dataset to apply clustering
    link_method (str): either "ward", "complete", "average", "single"
    max_nclus (int): maximum number of clusters to compare the methods
    min_nclus (int): minimum number of clusters to compare the methods. Defaults to 1.
    dist (str): distance to use to compute the clustering solution. Must be a valid distance. Defaults to "euclidean".
    
    Returns:
    ndarray: R2 values for the range of cluster solutions
    """
    def get_ss(df):
        ss = np.sum(df.var() * (df.count() - 1))
        return ss  # return sum of sum of squares of each df variable
    
    sst = get_ss(df)  # get total sum of squares
    
    r2 = []  # where we will store the R2 metrics for each cluster solution
    
    for i in range(min_nclus, max_nclus+1):  # iterate over desired ncluster range
        cluster = AgglomerativeClustering(n_clusters=i, affinity=dist, linkage=link_method)
        hclabels = cluster.fit_predict(df) #get cluster labels
        df_concat = pd.concat((df, pd.Series(hclabels, name='labels')), axis=1)  # concat df with labels
        ssw_labels = df_concat.groupby(by='labels').apply(get_ss)  # compute ssw for each cluster labels
        ssb = sst - np.sum(ssw_labels)  # remember: SST = SSW + SSB
        r2.append(ssb / sst)  # save the R2 of the given cluster solution
        
    return np.array(r2)

In [ ]:
#on 'type_covariance'-you have to insert with '', for example 'full'
def gmm_components(data,type_covariance,min_components,max_components):
    # Selecting number of components based on AIC and BIC
    n_components = np.arange(min_components,max_components)
    models = [GaussianMixture(n, covariance_type=type_covariance, n_init=10, random_state=1).fit(data)
              for n in n_components]

    bic_values = [m.bic(data) for m in models]
    aic_values = [m.aic(data) for m in models]
    plt.plot(n_components, bic_values, label='BIC')
    plt.plot(n_components, aic_values, label='AIC')
    plt.legend(loc='best')
    plt.xlabel('n_components')
    plt.xticks(n_components)
    plt.show()

### DataSet for clustering by value

In [ ]:
df = pd.read_csv(r'C:\Users\classroom\OneDrive - NOVAIMS\BCWDS\BC5\values_units.csv', usecols=['ProductName_ID','Point-of-Sale_ID','Value'])

In [ ]:
table = pd.pivot_table(df, values='Value', index=['Point-of-Sale_ID'],
                    columns=['ProductName_ID'], aggfunc=np.sum)

In [ ]:
table = table.fillna(0)

In [ ]:
#Normalize the dataset to run the clustering methods
names = table.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(table)
scaled_df = pd.DataFrame(scaled_df, columns=names)
  
# Normalizing the Data 
normalized_data = normalize(scaled_df) 
  
# Converting the numpy array into a pandas DataFrame 
normalized_data = pd.DataFrame(normalized_data,columns=names) 

## DataSet for clustering by product preference

In [ ]:
df2 = pd.read_csv(r'C:\Users\classroom\OneDrive - NOVAIMS\BCWDS\BC5\values_units.csv', usecols=['ProductName_ID','Point-of-Sale_ID','Units'])

In [ ]:
table2 = pd.pivot_table(df2, values='Units', index=['Point-of-Sale_ID'],
                    columns=['ProductName_ID'], aggfunc=np.sum)

In [ ]:
table2 = table2.fillna(0)

In [ ]:
#Normalize the dataset to run the clustering methods
names = table2.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(table2)
scaled_df = pd.DataFrame(scaled_df, columns=names)
  
# Normalizing the Data 
normalized_data2 = normalize(scaled_df) 
  
# Converting the numpy array into a pandas DataFrame 
normalized_data2 = pd.DataFrame(normalized_data2,columns=names) 

### Clustering for value

KMeans


K-Elbow plots

In [ ]:
number_clusters(normalized_data ,KMeans( init='k-means++', n_init=15, random_state=1))

In [ ]:
[SilhouetteVisualizer(KMeans(n_clusters=i,init='k-means++', n_init=15, random_state=1), colors='yellowbrick').fit(normalized_data).show() for i in range(2,7)]

In [ ]:

#According to the KElbowVisualizer and the SilhouetteVisualizer we were able to determine that the optimal number of clusters is 4.
#confirmar nº de k
kmclust = KMeans(n_clusters=3, init='k-means++', n_init=15, random_state=1)
km_labels = kmclust.fit_predict(normalized_data)
df_concat = pd.concat((normalized_data, pd.Series(km_labels, name='labels')), axis=1)
df_concat.groupby('labels').mean()

In [ ]:
#Making a copy of the dataset and appending a column with the cluster assigned to the observation.
km_df = normalized_data.copy()
km_df["km_labels"] = km_labels
km_df

GMM

Choosing the type of covariance

In [ ]:
gmm_config(normalized_data,10)

For the most part, the number of components the model with the lowest AIC and BIC is 'diag' 


Applying the clustering

In [ ]:
# Performing GMM clustering
gmm = GaussianMixture(n_components=3, covariance_type='diag', n_init=10, init_params='kmeans', random_state=1)
gmm_labels = gmm.fit_predict(normalized_data)
# Concatenating the labels to df
df_concat = pd.concat([normalized_data, pd.Series(gmm_labels, index=normalized_data.index, name="gmm_labels")], axis=1)
df_concat.head()

In [ ]:
gmm_ = normalized_data.copy()
gmm_["gmm_labels"] = gmm_labels
gmm_

### Hierarquical Clustering

In [ ]:
# Performing HC
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', n_clusters=5)
hc_labels = hclust.fit_predict(normalized_data)
hc_labels

In [ ]:
# Computing SST
X = normalized_data.values
sst = np.sum(np.square(X - X.mean(axis=0)), axis=0)

# Computing SSW
ssw_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssw_iter.append(np.sum(np.square(X_k - X_k.mean(axis=0)), axis=0))
ssw = np.sum(ssw_iter, axis=0)

# Computing SSB
ssb_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssb_iter.append(X_k.shape[0] * np.square(X_k.mean(axis=0) - X.mean(axis=0)))
ssb = np.sum(ssb_iter, axis=0)

# Verifying the formula
np.round(sst) == np.round((ssw + ssb))

In [ ]:
# Prepare input
hc_methods = ["ward", "complete", "average", "single"]
# Call function defined above to obtain the R2 statistic for each hc_method
max_nclus = 10
r2_hc_methods = np.vstack([get_r2_hc(df=normalized_data, link_method=link, max_nclus=max_nclus) for link in hc_methods]).T
r2_hc_methods = pd.DataFrame(r2_hc_methods, index=range(1, max_nclus + 1), columns=hc_methods)

sns.set()
# Plot data
fig = plt.figure(figsize=(11,5))
sns.lineplot(data=r2_hc_methods, linewidth=2.5, markers=["o"]*4)

# Finalize the plot
fig.suptitle("R2 plot for various hierarchical methods", fontsize=21)
plt.gca().invert_xaxis()  # invert x axis
plt.legend(title="HC methods", title_fontsize=11)
plt.xticks(range(1, max_nclus + 1))
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R2 metric", fontsize=13)

plt.show()

In [ ]:
# setting distance_threshold=0 and n_clusters=None ensures we compute the full tree
linkage = 'complete'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, affinity=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(normalized_data)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 1.57
dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'{distance.title()} Distance', fontsize=13)
plt.show()

In [ ]:
# 3 cluster solution
linkage = 'average'
distance = 'euclidean'
hc3lust = AgglomerativeClustering(linkage=linkage, affinity=distance, n_clusters=3)
hc3_labels = hc3lust.fit_predict(normalized_data)

In [ ]:
# Characterizing the 3 clusters
HC= pd.concat((normalized_data, pd.Series(hc3_labels, name='labels')), axis=1)
HC.groupby('labels').mean()

#### Metrica para comparar clusters

In [ ]:
def get_ss(df):
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable
def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst

In [ ]:
r2_list = [r2(km_df, 'km_labels'), r2(gmm_, 'gmm_labels'), r2(HC, 'labels')]
r2_tograph = np.asarray(r2_list)
# Create a List of Labels for x-axis
names = ["K-Means", "Gmm","HC"]
# Make the Chart
plt.bar(names, r2_tograph, color=['black', 'red', 'green'])
plt.title('Comparison of the different clustering methods')
plt.xlabel('Clustering method')
plt.ylabel('R2')
# Show the Chart
plt.show()


print('The r2 of the k-means method is', r2(km_df, 'km_labels').round(2))
print('The r2 of the GMM method is', r2(gmm_, 'gmm_labels').round(2))
print('The r2 of the HC method is', r2(HC, 'labels').round(2))

Due the fact that the all cluster methods have the same number of clusters we can choose the best method by comparing the r^2 of each method
After analyse the previous barplot we came to the conclusion that the HC is the best method

In [ ]:
#Clusters to analyse
table = table.reset_index()
HC_final = pd.concat((table , pd.Series(hc3_labels, name='labels')), axis=1)
HC_final = HC_final.set_index('Point-of-Sale_ID')
HC_final

In [ ]:
#Saving the clusters to analyse later
cluster10 = HC_final.loc[HC_final['labels'] == 0]
cluster11 = HC_final.loc[HC_final['labels'] == 1]
cluster12 = HC_final.loc[HC_final['labels'] == 2]


#check if we didn't lost any POS
len(HC_final) == len(cluster11) + len(cluster12)  +len(cluster10)

In [ ]:
#save the clusters in a csv to use later 

cluster10.to_csv('cluster10.csv', index=False)
cluster11.to_csv('cluster11.csv', index=False)
cluster12.to_csv('cluster12.csv', index=False)

### Clustering for product

KMeans

K-Elbow plots

In [ ]:
number_clusters(normalized_data2 ,KMeans( init='k-means++', n_init=15, random_state=1))

In [ ]:
[SilhouetteVisualizer(KMeans(n_clusters=i,init='k-means++', n_init=15, random_state=1), colors='yellowbrick').fit(normalized_data2).show() for i in range(2,7)]

In [ ]:
#According to the KElbowVisualizer and the SilhouetteVisualizer we were able to determine that the optimal number of clusters is 4.
kmclust = KMeans(n_clusters=4, init='k-means++', n_init=15, random_state=1)
km_labels = kmclust.fit_predict(normalized_data2)
df_concat = pd.concat((normalized_data2, pd.Series(km_labels, name='labels')), axis=1)
df_concat.groupby('labels').mean()

In [ ]:
#Making a copy of the dataset and appending a column with the cluster assigned to the observation.
km_df2 = normalized_data.copy()
km_df2["km_labels"] = km_labels
km_df2

In [ ]:
GMM

Choosing the type of covariance

In [ ]:
gmm_config(normalized_data2,10)

For the most part, the number of components the model with the lowest AIC and BIC is 'diag' 


Find the number of components

In [ ]:
gmm_components(normalized_data2,'diag',1,10)

According to the graph, the teams considers the optimal number of components is 5.

Applying the clustering

In [ ]:
# Performing GMM clustering
gmm = GaussianMixture(n_components=5, covariance_type='diag', n_init=10, init_params='kmeans', random_state=1)
gmm_labels = gmm.fit_predict(normalized_data)
# Concatenating the labels to df
df_concat = pd.concat([normalized_data, pd.Series(gmm_labels, index=normalized_data.index, name="gmm_labels2")], axis=1)
df_concat.head()

In [ ]:
gmm_2= normalized_data.copy()
gmm_2["gmm_labels2"] = df_concat['gmm_labels2']
gmm_2

## HC

In [ ]:
# Performing HC
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', n_clusters=5)
hc_labels = hclust.fit_predict(normalized_data2)
hc_labels

In [ ]:
# Computing SST
X = normalized_data2.values
sst = np.sum(np.square(X - X.mean(axis=0)), axis=0)

# Computing SSW
ssw_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssw_iter.append(np.sum(np.square(X_k - X_k.mean(axis=0)), axis=0))
ssw = np.sum(ssw_iter, axis=0)

# Computing SSB
ssb_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssb_iter.append(X_k.shape[0] * np.square(X_k.mean(axis=0) - X.mean(axis=0)))
ssb = np.sum(ssb_iter, axis=0)

# Verifying the formula
np.round(sst) == np.round((ssw + ssb))

In [ ]:
# Prepare input
hc_methods = ["ward", "complete", "average", "single"]
# Call function defined above to obtain the R2 statistic for each hc_method
max_nclus = 10
r2_hc_methods = np.vstack([get_r2_hc(df=normalized_data2, link_method=link, max_nclus=max_nclus) for link in hc_methods]).T
r2_hc_methods = pd.DataFrame(r2_hc_methods, index=range(1, max_nclus + 1), columns=hc_methods)

sns.set()
# Plot data
fig = plt.figure(figsize=(11,5))
sns.lineplot(data=r2_hc_methods, linewidth=2.5, markers=["o"]*4)

# Finalize the plot
fig.suptitle("R2 plot for various hierarchical methods", fontsize=21)
plt.gca().invert_xaxis()  # invert x axis
plt.legend(title="HC methods", title_fontsize=11)
plt.xticks(range(1, max_nclus + 1))
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R2 metric", fontsize=13)

plt.show()

In [ ]:
# setting distance_threshold=0 and n_clusters=None ensures we compute the full tree
linkage = 'ward'
distance = 'euclidean'
hclust = AgglomerativeClustering(linkage=linkage, affinity=distance, distance_threshold=0, n_clusters=None)
hclust.fit_predict(normalized_data2)

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

# create the counts of samples under each node (number of points being merged)
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)

# hclust.children_ contains the observation ids that are being merged together
# At the i-th iteration, children[i][0] and children[i][1] are merged to form node n_samples + i
for i, merge in enumerate(hclust.children_):
    # track the number of observations in the current cluster being formed
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            # If this is True, then we are merging an observation
            current_count += 1  # leaf node
        else:
            # Otherwise, we are merging a previously formed cluster
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count

# the hclust.children_ is used to indicate the two points/clusters being merged (dendrogram's u-joins)
# the hclust.distances_ indicates the distance between the two points/clusters (height of the u-joins)
# the counts indicate the number of points being merged (dendrogram's x-axis)
linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)

# Plot the corresponding dendrogram
sns.set()
fig = plt.figure(figsize=(11,5))
# The Dendrogram parameters need to be tuned
y_threshold = 2.75
dendrogram(linkage_matrix, truncate_mode='level', p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - {linkage.title()}\'s Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'{distance.title()} Distance', fontsize=13)
plt.show()

In [ ]:
# 3 cluster solution
linkage = 'average'
distance = 'euclidean'
hc3lust = AgglomerativeClustering(linkage=linkage, affinity=distance, n_clusters=4)
hc3_labels = hc3lust.fit_predict(normalized_data2)

In [ ]:
# Characterizing the 3 clusters
HC2= pd.concat((normalized_data, pd.Series(hc3_labels, name='labels')), axis=1)
HC2.groupby('labels').mean()

### Metrica para comparar clusters

In [ ]:
r_list = [r2(km_df2, 'km_labels'), r2(gmm_2, 'gmm_labels'), r2(HC2, 'labels')]
r_tograph = np.asarray(r_list)
# Create a List of Labels for x-axis
names = ["K-Means", "Gmm","HC"]
# Make the Chart
plt.bar(names, r_tograph, color=['black', 'red', 'green'])
plt.title('Comparison of the different clustering methods')
plt.xlabel('Clustering method')
plt.ylabel('R2')
# Show the Chart
plt.show()

print('The r2 of the k-means method is', r2(km_df2, 'km_labels').round(2))
print('The r2 of the GMM method is', r2(gmm_2, 'gmm_labels').round(2))
print('The r2 of the HC method is', r2(HC2, 'labels').round(2))


Due the fact that the all cluster methods have the same number of variables we can choose the best method by comparing the r^2 of each method After analyse the previous barplot we came to the conclusion that the K-Means is the best method

In [ ]:
#Clusters to analyse
#table2 = table2.reset_index()
final_cluster20 = pd.concat((table2 , pd.Series(gmm_labels, name='gmm_labels')), axis=1)
final_cluster20 = final_cluster20.set_index('Point-of-Sale_ID')
final_cluster20 = final_cluster20.drop(['index'], axis=1)
final_cluster20

In [ ]:
#Saving the clusters to analyse later
cluster20 = final_cluster20.loc[final_cluster20['gmm_labels'] == 0]
cluster21 = final_cluster20.loc[final_cluster20['gmm_labels'] == 1]
cluster22 = final_cluster20.loc[final_cluster20['gmm_labels'] == 2]
cluster23 = final_cluster20.loc[final_cluster20['gmm_labels'] == 3]
cluster24 = final_cluster20.loc[final_cluster20['gmm_labels'] == 4]


In [ ]:
#save the clusters to a csv to load lately
cluster20.to_csv('cluster20.csv', index=False)
cluster21.to_csv('cluster21.csv', index=False)
cluster22.to_csv('cluster22.csv', index=False)
cluster23.to_csv('cluster23.csv', index=False)
cluster24.to_csv('cluster24.csv', index=False)


### Clusters Analises

In [ ]:
#value
LowValuePOS = pd.read_csv(r'cluster10.csv')
MediumValuePOS = pd.read_csv(r'cluster11.csv')
HighValuesPOS = pd.read_csv(r'cluster12.csv')

#Units
cluster20 = pd.read_csv(r'cluster20.csv')
cluster21 = pd.read_csv(r'cluster21.csv')
cluster22 = pd.read_csv(r'cluster22.csv')
cluster23 = pd.read_csv(r'cluster23.csv')
cluster24 = pd.read_csv(r'cluster24.csv')


## Per value
Distribution of the number of pos in each cluster

In [ ]:
#clusters distribution
list1 = [len(LowValuePOS),len(MediumValuePOS),len(HighValuesPOS)]
list1 = np.asarray(list1)
# Create a List of Labels for x-axis
names = ["LowValuePOS", "MediumValuePOS","HighValuesPOS"]
# Make the Chart
plt.bar(names, list1)
plt.title('clusters distribution')
plt.xlabel('Cluster')
plt.ylabel('Total of POS in each cluster')
# Show the Chart
plt.show()

In [ ]:
print('The medium value of sells of the POS in the LowValuePOS are', LowValuePOS.mean().mean(), 'and the total value is', LowValuePOS.sum().sum())

In [ ]:
print('The medium value of sells of the POS in the LowValuePOS are', MediumValuePOS.mean().mean(), 'and the total value is', MediumValuePOS.sum().sum())

In [ ]:
print('The medium value of sells of the POS in the LowValuePOS are', HighValuesPOS.mean().mean(), 'and the total value is', HighValuesPOS.sum().sum())

## Per product preference

Distribution of the number of pos in each cluster

In [ ]:

#clusters distribution
list1 = [len(cluster20),len(cluster21),len(cluster22),len(cluster23),len(cluster24)]
list1 = np.asarray(list1)
# Create a List of Labels for x-axis
names = ["Cluster 0", "Cluster 1","Cluster 2", 'Cluster 3', 'Cluster 4']
# Make the Chart
plt.bar(names, list1)
plt.title('clusters distribution')
plt.xlabel('Cluster')
plt.ylabel('Total of POS in each cluster')
# Show the Chart
plt.show()

In [ ]:
cluster20.sum().sort_values(ascending= False).head(10)

In [ ]:
cluster21.sum().sort_values(ascending= False).head(10)

In [ ]:
cluster22.sum().sort_values(ascending= False).head(10)

In [ ]:
cluster23.sum().sort_values(ascending= False).head(10)

In [ ]:
cluster24.sum().sort_values(ascending= False).head(10)

# Forecast